Alice and Bob play the following game:

- Alice chooses a 10-letter word from a fixed dictionary, uniformly at random
- At each turn Bob proposes a word from the same dictionary
- For each proposal, Alice returns the "score" of the proposed word: each correct letter gives 1 point, plus an additional point if it is at the right position.
- The total score of Bob is the sum of scores over k rounds with k fixed to 5, 10 or 15.

The goal is to maximize the score of Bob over 100 games  - where each game lasts k rounds - (or its expectation). The score is then normalized so that the maximum score is 100.

You are allowed to use reasonable precomputation (less than one hour) and your code should play any game in a reasonable amount of time (less than a few minutes). Running time will also be considered in grading. You can use any library, even if it is not installed. To install a library with `pip`, just paste the command with `!` before: `!pip install numpy` (this one is already installed).

Create a copy of this notebook that you will put in the Google form. Runtime > Run all should execute your code without error. If you have suggestions for improvements, feel free to add them and we might take them into account in the evaluation.

## Testing code

Execute the code below to download the word list and define the testing class.
It is recommended to read this code. 

**DO NOT MODIFY ANYTHING IN THIS SECTION**

In [ ]:
!wget -nc https://gematria.tech/words_10.txt 2> /dev/null

In [ ]:
from collections import Counter

def score(word1, word2):
    """scoring function
    """
    assert len(word1) == len(word2)
    ans = 0
    hist = Counter(word1)
    for c in word2:
        if hist[c]:
            hist[c] -= 1
            ans += 1
    for c1, c2 in zip(word1, word2):
        ans += c1 == c2
    return ans

assert score("abc", "abd") == 4
assert score("abc", "bca") == 3 

In [ ]:
import random
from tqdm.autonotebook import tqdm

class Simulator:
    def __init__(self, path="words_10.txt"):
        with open(path) as f:
            self.words = list(map(str.strip, f))
            self.words_set = self.words

    def play(self, bob, rounds):
        total_score = 0
        word = random.choice(self.words)
        bob.new_game()
        for _ in range(rounds):
            guess = bob.guess()
            if guess not in self.words_set:
                return 0
            s = score(word, guess)
            bob.clue(s)
            total_score += s
        return total_score

    def score(self, bob, games, rounds):
        ans = sum(self.play(bob, rounds) for _ in tqdm(range(games)))
        return 10_000 * ans / games / rounds // 20 / 100 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [ ]:
class ExampleBob:
    """
    You should copy and modify this code to implement your solution
    """
    def __init__(self, words, rounds):
        """
        Call this only once to create your player. Any precomputation should go here.
        """
        self.words = words
        self.rounds = rounds
        
    def new_game(self):
        """
        Prepare for a new game
        """
        pass

    def guess(self):
        """
        Make a new guess
        """
        return random.choice(self.words)

    def clue(self, score):
        """
        The last guess got this score.
        """
        pass


In [ ]:
from time import time

simulator = Simulator()

for rounds in [5, 10, 15]:
    t = time()
    bob = ExampleBob(simulator.words.copy(), rounds)
    print('%.02f s' % (time() - t))
    print("score", simulator.score(bob, games=100, rounds=rounds))


FileNotFoundError: ignored

## Your code

This is the only section you should modify. You can add any number of cells. Comments are welcome but not necessary.

In [ ]:
from tqdm.autonotebook import tqdm
## was planning to use this as a preproccessing but realised they're only 100 games and i don;t hve time atm
path="words_10.txt"
with open(path) as f:
  words = list(map(str.strip, f))

mat =[]
w1 = words
w2 = words

for i in w1:
  temp = []
  for j in w2:
    temp.append(score(i,j)) 
  mat.append(temp)

for row in mat:
  print(row)

"""
Creating a 2D lookup table to try to get the values determining the actual word and reduce the number of needed geusses to find a word
"""

In [ ]:
class Bob:
    """
    You should only modify this code
    """
    
    def __init__(self, words, rounds):
        """
        Call this only once to create your player. Any precomputation should go here.
        """
        self.words = words
        self.rounds = rounds
        ##self.createMatrix()
       

    def new_game(self):
      ## initialising variables that are needed for every new word
        self.old="          "
        self.score = 0
        self.curr = "          "
        self.guesses = ["          "]
        self.colsOld = []
        self.colsNew = []
        ##print("NEW GAME")
        pass
    
    """def createMatrix(self):##hidden due to very large run time o(n^2)
        self.mat =[]
        w1 = self.words
        w2 = self.words
        print(w1)
        for i in w1:
          temp = []
          for j in w2:
              temp.append(score(i,j)) 
          self.mat.append(temp)"""

    def createScore(self,x): ## o(n) get the score for all the words that with a specific word
        w1 = self.words[x]
        w2 = self.words
        ##print(len(w2))
        temp = []
        for j in w2:
            temp.append(score(w1,j)) 
        return temp
    
    

    """def guess(self):
    
        Make a new guess
         
        ## Guess the word and try to find a second geuss that is similar to the score you gt from the simulation to improve the chaces of finding the best word
        test = random.choice(self.words)
        flag = True
        while(flag):    
          while(test in self.guesses):
            test = random.choice(self.words)
          count = score(self.curr, test)
          
          if(count == self.score):
            flag = False
          test = random.choice(self.words)

        self.rounds+=1

        ##print(count, self.score)
        ##print(test)
        
        self.old = self.curr
        self.curr = test
        self.guesses.append(self.curr)
        return self.curr
  """
    def intersection(self,lst1, lst2):##https://www.geeksforgeeks.org/python-intersection-two-lists/ gets the intersection of 2 lists
      lst3 = [value for value in lst1 if value in lst2]
      return lst3

    def GetIndex(self,lis, desired):##https://www.geeksforgeeks.org/python-duplicate-element-indices-in-list/ gets the index of variable des in a list lis
      test_list = lis
      des = desired
      oc_set = set()
      res = []
      for idx, val in enumerate(test_list):
        if val == des:
          res.append(idx)     
      return res

    def guess(self): # my take on the guessing style 
      
      if(self.curr == "          "): # if first time guess randomly
        test = random.choice(self.words)
        self.curr = test
        self.guesses.append(self.curr)
        ##print("first Guess: ",test)
        return test
      # for an iterative guess get the index of the guessed word
      row = self.words.index(self.curr) 
      # then take that row and find the words that get the same scores when tested with the guessed word and put then in a new list colNew
      self.colsNew = self.GetIndex(self.createScore(row),self.score) ## get the words that might equte to that same score
      if self.score == 20: # if correct word found don't change anything
        self.colsNew = [row]
      if not self.colsOld: ## if old list is empty initialise it properly for the first iteration
        ##print("I'm empty")
        self.colsOld = self.colsNew.copy()

      ##print(self.colsOld)
      ##print(self.colsNew)
      ##print(self.score)

      ##if self.score < 20 and len(self.colsNew)>0:
      self.colsOld = self.intersection(self.colsOld, self.colsNew) # get the intersection of the first list and the secnd list to reduce the number of right possibe guesses
      
      wop = self.colsOld[0]
      test = self.words[wop]
      ##print(self.colsOld)
      ##print("Geussing:",test)
      ##print("-----------------------------------------------------------------")
      
      self.curr = test
      self.guesses.append(self.curr)
      return test

    def clue(self, score):
        """
        The last guess got this score.
        """
        self.score = score
        """if self.score > score: #old way of thinking NVM this
          ##print(self.score, ">", score)
          self.curr = self.old  
          pass
        if self.score <= score:
          ##print(self.score, "<", score)
          self.score = score """
        pass


# Evaluation

**DO NOT MODIFY**

In [ ]:
from time import time

simulator = Simulator()

for rounds in [5, 10, 15]:
    t = time()
    bob = Bob(simulator.words.copy(), rounds)
    print('%.02f s' % (time() - t))
    print("score", simulator.score(bob, games=100, rounds=rounds))


In [ ]:
mat = []
w1 = ["boy","gir", "eat", "dri"]
w2 = ["boy","gir", "eat", "dri"]
for i in w1:
  temp = []
  for j in w2:
      temp.append(score(i,j)) 
  mat.append(temp)
for row in mat:
  print(row)

In [ ]:

test_list = [1, 4, 5, 5, 5, 9, 1]
des = 5
  

oc_set = set()
res = []
for idx, val in enumerate(test_list):
             
    if val == des:
        res.append(idx)     
          
# printing result
print(res)